In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\spark-3.4.0-bin-hadoop3'

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Database-join").getOrCreate()

In [3]:
joinA_rdd = spark.sparkContext.textFile(r"C:\Users\Gurudeep\Desktop\spark-warehouse\joinA.txt")
joinB_rdd = spark.sparkContext.textFile(r"C:\Users\Gurudeep\Desktop\spark-warehouse\joinB.txt")

In [9]:
joinA_rdd.first()

'able,991'

In [70]:
joinA_pairs_rdd = joinA_rdd.map(lambda line: line.split(",")).map(lambda cols: (cols[0],cols[1]))
joinB_pairs_rdd = joinB_rdd.map(lambda line: line.split(",")).map(lambda cols: (cols[0].split()[1],cols[0].split()[0],cols[1]))
#joinB_pairs_rdd = joinB_rdd.map(lambda line: line.split(",")).map(lambda cols: (cols[0],cols[1]))

In [71]:
print(joinA_pairs_rdd.take(5))
print(joinB_pairs_rdd.take(8))

[('able', '991'), ('about', '11'), ('burger', '15'), ('actor', '22'), ('about', '13')]
[('able', 'Jan-01', '5'), ('about', 'Feb-02', '3'), ('about', 'Mar-03', '8'), ('able', 'Apr-04', '13'), ('actor', 'Feb-22', '3'), ('burger', 'Feb-23', '5'), ('burger', 'Mar-08', '2'), ('able', 'Dec-15', '100')]


In [36]:
dataColl=joinB_pairs_rdd.collect()
for row in dataColl:
    print(row[0]," , ",row[1]," , ",row[2])

able  ,  Jan-01  ,  5
about  ,  Feb-02  ,  3
about  ,  Mar-03  ,  8
able  ,  Apr-04  ,  13
actor  ,  Feb-22  ,  3
burger  ,  Feb-23  ,  5
burger  ,  Mar-08  ,  2
able  ,  Dec-15  ,  100


In [72]:
joined_rdd = joinB_pairs_rdd.join(joinA_pairs_rdd)

In [86]:
df_1 = joinA_pairs_rdd.toDF(['col1','col2'])
df_2 = joinB_pairs_rdd.toDF(['col_1','col_2','col_3'])

In [89]:
joined_df = df_1.join(df_2,df_2.col_1 == df_1.col1).select("col1","col_2","col_3","col2")

In [90]:
joined_df.show()

+------+------+-----+----+
|  col1| col_2|col_3|col2|
+------+------+-----+----+
|  able|Jan-01|    5| 991|
|  able|Apr-04|   13| 991|
|  able|Dec-15|  100| 991|
| about|Feb-02|    3|  11|
| about|Mar-03|    8|  11|
| about|Feb-02|    3|  13|
| about|Mar-03|    8|  13|
| actor|Feb-22|    3|  22|
|burger|Feb-23|    5|  15|
|burger|Mar-08|    2|  15|
+------+------+-----+----+



In [93]:
import pandas as pd

In [96]:
df = pd.DataFrame()

In [94]:
col1 = joined_df.select("col1").rdd.flatMap(lambda x: x).collect()
col_2 = joined_df.select("col_2").rdd.flatMap(lambda x: x).collect()
col_3 = joined_df.select("col_3").rdd.flatMap(lambda x: x).collect()
col2 = joined_df.select("col2").rdd.flatMap(lambda x: x).collect()

In [97]:
df['col1'] = col1
df['col_2'] = col_2
df['col_3'] = col_3
df['col2'] = col2

In [101]:
# create tuples for each row
tuples = list(df.itertuples(index=False, name=None))

# write tuples to text file
with open('output.txt', 'w') as f:
    for t in tuples:
        line = '(' + ','.join(str(x) for x in t) + ')' + '\n'
        f.write((line))